In [1]:
# to be added on views.py

import pickle

loaded_model = pickle.load(open('rfa_model.sav', 'rb'))
loaded_model

extractor = pickle.load(open('vectorizer.sav', 'rb'))
print(extractor)

CountVectorizer(min_df=0,
                stop_words=['(l)', '(polynomial)', '(w)',
                            'add(mul(integer((d))pow(symbol(x)integer((d))))symbol(x)mul(integer(-(d))mul(integer((d))integer((d))pow(mul(integer((d))symbol(x))mul(integer((d))pow(integer((d))integer(-(d))))))))',
                            'add(mul(integer(-(d))integer((d))pow(integer((d))mul(integer((d))pow(integer((d))integer(-(d))))))in...
                            'add(symbol(x)mul(integer(-(d))mul(integer((d))integer((d))symbol(x))))',
                            'add(symbol(x)symbol(x))', 'advised', 'allowance',
                            'answered', 'any)', 'asked', 'bank', 'blank', 'box',
                            'buy', 'cardboard', 'centimeter', 'centimeters',
                            'chain', 'christmas', 'classmate', 'classroom',
                            'closely', 'cm', 'composed', 'consider', 'constant',
                            'construction', 'contains', ...],
           

In [2]:
# To be added on views.py on webapp


from latex2sympy2 import latex2sympy
from sympy import *
from sympy.parsing.sympy_parser import parse_expr
import re


#Changes Before Sympy
def change_to_dollar(exp):
    exp = re.sub("\.\.\.|\\dots|…","x", exp)
    exp = re.sub("=","$ $", exp)
    return exp

def expression_checker(question):
    qs = question
    qs = re.sub(r"\\sqrt\{(.+?)\}", r"(\1)^{\\frac{1}{2}}", qs)
    qs = re.sub(r"[\\][(][)][\\]", "$", qs)
    fx = re.search('[$].+?[=].+?[$]',qs)
    len_check = len(re.findall('[$].+?[=].+?[$]', qs))
    iplus = 0
    while iplus<len_check:
        iplus += 1
        qs = qs[:fx.span()[0]] + " " + change_to_dollar(fx.group()) + " " + qs[fx.span()[1]:]
        #check for the next LaTex Enclosure if any         
        fx = re.search('[$].+?[=].+?[$]', qs)
    #the new question is added to index i       
    return qs

def sympify(question):
    test = question
    #regex value for searching all characters from '$' to the next '$' symbol
    check = re.search('[$].+?[$]',test)
    #checks how many instance of latex enclosure are there in the question     
    check_len = len(re.findall('[$].+?[$]',test))
    iteration = 0
    while iteration<check_len:
        iteration += 1
        print(check.group())
        test = test[:check.span()[0]] + " " + re.sub(" ","",str(latex2sympy(r"{}".format(check.group())))) + " " + test[check.span()[1]:]   
        check = re.search('[$].+?[$]',test)       
    return test

# Change Must Happen After Sympy 
def change_characters(question):   
    question = re.sub(' \+ ', '+',question)
    question = re.sub(' - ', '-',question)
    question = re.sub(' \+', '+',question)
    question = re.sub(' -', '-',question)
    question = re.sub('\+ ', '+',question)
    question = re.sub('- ', '-',question)
    question = re.sub('=', ' ',question)
    question = re.sub('\n','',question)
    question = re.sub('\?','',question)
    question = re.sub('\'','',question)
    question = re.sub(',','',question)
    question = re.sub('[\.]','',question)
    question = re.sub('\t', ' ',question)
    question = re.sub(r"\_\_+", "__",question)
    question = re.sub('[;]', ' ',question)
    question = re.sub(':', ' ',question)
    return question
    

def tokenize_formula(expression):
    try:
        return srepr(parse_expr(expression, evaluate=False))
    except:
        return expression

def expression_treefy(question):
    item_question = question
    tokenize_question_item = item_question.split()
    for x in range(len(tokenize_question_item)):
        check = re.findall('[\+]|[\*]|[-]|[/]|[0-9]',tokenize_question_item[x])
        if len(tokenize_question_item[x]) == 1:
            check = True 
        if check: 
            print(tokenize_question_item[x])
            text = re.sub("\s|[,]","",str(tokenize_formula(tokenize_question_item[x])))
            print(text)
            text = re.sub("\d+","(d)",text)
            text = re.sub("[(]['].+?['][)]","(x)",text)
#             text = re.sub("Symbol[(]['].+?['][)]","symbol(x)",text)
            text = re.sub('\s+', '', text)
            tokenize_question_item[x] = text
    item_question = ' '.join(tokenize_question_item)
    return item_question


# Token from Django have '\(' '\)' enclosures so that HTML can render LaTex Expression
# We convert them to $$ enclosure because of old dataset that we have before mathquill was applied to the system
# In this classify_topic function we did not include that function so we only evalue questions that are already enclosed by 
# $ $ sign

def classify_topic(cleaned_question):
    question = cleaned_question
    question = expression_checker(question)
    print("Step 1: Removing Equal Sign Inside LaTex So That Sympy would not evaluate the equation or answer the equation")
    print(question)
    question = sympify(question)
    print("Step 2: Converting LaTex Expressions into Freeform Structure")
    print(question)
    question = change_characters(question)
    print("Step 3: Removing Unnecessary Characters")
    print(question)
    question = expression_treefy(question)
    print("Step 4: Expressing Freeform Equations into Expression trees")
    print(question)
    vector = extractor.transform([question])
    print("Step 5: vectorizing the question based on the dataset")
    print(vector)
    topic = loaded_model.predict(vector)
    print("Step 6: Classifying the question")
    return topic

# it is advised that equation written are in LaTex or Freeform structure

test = r"it is an ordered set of pattern or rule (a1, a2,...)"
topic = classify_topic(test)
print(topic)


Step 1: Removing Equal Sign Inside LaTex So That Sympy would not evaluate the equation or answer the equation
it is an ordered set of pattern or rule (a1, a2,...)
Step 2: Converting LaTex Expressions into Freeform Structure
it is an ordered set of pattern or rule (a1, a2,...)
Step 3: Removing Unnecessary Characters
it is an ordered set of pattern or rule (a1 a2)
(a1
(a1
a2)
a2)
Step 4: Expressing Freeform Equations into Expression trees
it is an ordered set of pattern or rule (a(d) a(d))
Step 5: vectorizing the question based on the dataset
  (0, 506)	1
  (0, 511)	1
  (0, 539)	1
Step 6: Classifying the question
['series_and_sequence']
